In [ ]:
import tensorflow as tf
from tensorflow import keras
import sklearn
import pandas
import numpy as np

In [ ]:
# the name of the data file
file_name = 'Dataset\\Data.xlsx'
# the forecast size is the number of points that will be forecasted
# That means How many days or hours or months you want forecast , if the data monthly and put in forecast size 15 then code will display forecast for 15 months  
forecast_size = 15
# the number of the category that the model will train on (open) hase number 1, (close) has number 4 .
# That means forecast based which category you want .
category = 3
# the percentage of the data that will be used for training. the value of must be between 0.0, 1.0
train_percentage = 0.8

In [ ]:
#Reading the excel file
df = pandas.read_excel(file_name)
keys = [i for i in df]
data = df.to_numpy()[1:]
train_size = int(data.shape[0] * train_percentage)

In [ ]:
#Detection the type of input dataset to do the later prediction (yearly,monthly,weekly,daily,hourly)
monthly, daily, hourly, weekly,yearly = 0, 0, 0, 0, 0
aa = data[1][0]-data[0][0]
if aa == 1:
    yearly = 1
    period = 1
else:
    aa = aa.value
    if aa / 3600000000000 == 1:
        hourly = 1
        period = 24
    elif aa / 3600000000000 == 24:
        daily = 1
        period = 30
    elif aa / 3600000000000 == 24*7:    
        weekly = 1
        period = 1
    else:
        monthly = 1
        period = 12
        
print('hourly : ',hourly) 
print('daily  : ',daily)
print('weekly : ',weekly)
print('monthly: ',monthly)
print('yearly : ',yearly)
    

In [ ]:
#Splite the dataset
date = data[0,0]
data = np.float32(data[:,category])
data[np.isnan(data)] = 0
data[data==0] = 2**-15
normalize = 1
for i in range(16):
    if abs(np.max(data) - 2**i) < abs(np.max(data) - normalize):
        normalize = 2**i

data = data / normalize
x = []
y = []
for i in range(data.shape[0]-period):        
    x.append([data[i+j] for j in range(period)])
    y.append([data[i+period]])

x = np.array(x)
y = np.array(y)
x_train = x[:train_size]
y_train = y[:train_size]
if True:
    x.shape = (1,)+ x.shape
    y.shape = (1,)+ y.shape
    x_train.shape = (1,)+ x_train.shape
    y_train.shape = (1,)+ y_train.shape

In [ ]:
#creat model
tf.random.set_random_seed(0)    
k = keras.backend
def mape2(y_true, y_pred):
    er_train = keras.losses.mape(y_true, y_pred)
    er_test = keras.losses.mape(y_true[:,train_size:], y_pred[:,train_size:])
    er = (k.mean(er_train, axis=-1, keepdims=True) + k.mean(er_test, axis=-1, keepdims=True)) * 0.5
    return er

file_name = 'Results\\A_CNN_Data.xlsx'
# CNN
model = keras.Sequential([
keras.layers.InputLayer((None, x_train.shape[2])),
keras.layers.Conv1D(128, 3, strides=1, padding='same', activation='elu'),
keras.layers.Conv1D(128, 3, strides=1, padding='same', activation='elu'),
keras.layers.Conv1D(128, 3, strides=1, padding='same', activation='elu'),
keras.layers.Conv1D(128, 3, strides=1, padding='same', activation='elu'),
keras.layers.Conv1D(1, 1, strides=1, padding='same')
])
model.compile('adam', 'mse', ['mape', mape2])
model.summary()
model.fit(x_train, y_train, epochs=1000, validation_data=(x, y), 
          callbacks=[keras.callbacks.EarlyStopping(
                monitor='val_mape2', patience=4000,
                mode='min', restore_best_weights=True) ])

In [ ]:
# Test the model and calculate the error and creating the output file
mape = np.mean(np.abs(model.predict(x_train) - y_train) / np.abs(y_train)) * 100
print('train mape: ' + str(mape) + '\n')
p = model.predict(x)
if p.shape[0] == 1:
    mape = np.mean(np.abs(p[:,train_size:] - y[:,train_size:]) / np.abs(y[:,train_size:])) * 100
else:
    mape = np.mean(np.abs(p[train_size:] - y[train_size:]) / np.abs(y[train_size:])) * 100

print('test mape: ' + str(mape) + '\n')

o = x.copy()
shape = o.shape[0]
if shape == 1:
    shape = o.shape[1]
for i in range(shape, shape+forecast_size+1):
    p = model.predict(o)
    if o.shape[0] > 1:
        ap = np.append(o[-1:,1:], p[-1:], axis=-1)
    else:
        ap = np.append(o[:,-1:,1:], p[:,-1:], axis=-1)
    o = np.append(o, ap, axis=-2)

if o.shape[0] == 1: o = o[0]
oo = []
pad = [[i] for i in data[:period-1]]
for j in [pad, o]:
    for i in j:
        oo.append([date, i[-1]*normalize])
        if hourly == 1:
            date = date + pandas.offsets.DateOffset(hours=1)
        elif daily == 1:
            date = date + pandas.offsets.DateOffset(days=1)
        elif monthly == 1:
            date = date + pandas.offsets.DateOffset(months=1)
        elif weekly == 1:
            date = date + pandas.offsets.DateOffset(weeks=1)    
        elif yearly == 1:
            date = date + 1
            

df = pandas.DataFrame(oo)
df.columns = ['date', keys[category]]
pandas.set_option("display.max_rows", None, "display.max_columns", None)
print('forecasted data:')
print(df)
df.to_excel(file_name, index=False)

In [ ]:
def uniplot(df1,df2,df1_col1,df1_col2,df2_col2,xlable,ylabel,legend1,legend2):
    plt.figure(1, figsize=(14, 6))
    plt.plot(df1[df1_col1],df1[df1_col2],'r--')
    plt.plot(df2.iloc[:,0],df2[df2_col2],'b')
    plt.xlabel(xlable)
    plt.ylabel(ylabel)
    plt.legend([legend1, legend2])
    

In [ ]:
%matplotlib inline
import matplotlib. pyplot as plt


In [ ]:
df1 = pandas.read_excel('Results\\A_CNN_Data.xlsx', parse_dates=True)
df2 = pandas.read_excel('Dataset\\Data.xlsx', parse_dates=True)

In [ ]:
uniplot(df1,df2,'date','Low','Low','Date','Low','Predict','Actual')